# Long Term Memory
- Cross thread memory to store user-specific or application specific
- Useful for applications like chatbot to remember user preference

## Key Points
- Namespace
- embedding model

In [104]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

In [105]:
embeddings = init_embeddings(
    'openai: text-embedding-3-small'
)

In [106]:
store = InMemoryStore(
   index = { "embed": embeddings, 
    "dim": 1536}
)

In [107]:
namespace = ("user_1", "memories")

In [108]:
store.put(namespace, "1", {"text": " I love pizza"})

In [109]:
store.put(namespace, "2", {"text": "I prefer italina food"})

In [110]:
store.put(namespace, "3", {"text": "I dont like spicy food"})
store.put(namespace, "4", {"text": "I love japaneese food"})

In [111]:
memories = store.search(
    namespace, query="I like food", limit=4
)

In [112]:
memories

[Item(namespace=['user_1', 'memories'], key='2', value={'text': 'I prefer italina food'}, created_at='2025-06-12T12:24:37.510883+00:00', updated_at='2025-06-12T12:24:37.510896+00:00', score=0.4610445501208644),
 Item(namespace=['user_1', 'memories'], key='4', value={'text': 'I love japaneese food'}, created_at='2025-06-12T12:24:38.396728+00:00', updated_at='2025-06-12T12:24:38.396740+00:00', score=0.4510500287410169),
 Item(namespace=['user_1', 'memories'], key='3', value={'text': 'I dont like spicy food'}, created_at='2025-06-12T12:24:38.019889+00:00', updated_at='2025-06-12T12:24:38.019903+00:00', score=0.3952332248434571),
 Item(namespace=['user_1', 'memories'], key='1', value={'text': ' I love pizza'}, created_at='2025-06-12T12:24:36.897932+00:00', updated_at='2025-06-12T12:24:36.897948+00:00', score=0.37500770477215534)]

In [113]:
from typing import Optional
from langchain.chat_models import init_chat_model
from langgraph.store.base import BaseStore
from langgraph.graph import START, MessagesState, StateGraph


In [114]:
llm = init_chat_model("openai:gpt-4o-mini")

In [ ]:
def chat(state, *, store: BaseStore):
    msg = state["messages"][-1]
    print(f"My message or my query {msg}")

    if msg.content.startswith("remember"):
        text = msg.content[len("remember"):].strip()
        store.put(namespace, "6", {"text": text})

    items = store.search(namespace, query=msg.content, limit=5)
        
    memories = "\n".join( item.value["text"] for item in items)
    print(f"my memories {memories}")
    memories_prompt = f" ## Memories of user \n {memories}" if memories else ""
    response = llm.invoke(
        [
            {"role": "system", "content": f" you are a helpful assistant. \n {memories_prompt}"},
            *state["messages"]
        ]
    )
    return {"messages": [response]}


In [126]:
builder = StateGraph(MessagesState)
builder.add_node(chat)
builder.add_edge(START, "chat")


In [127]:
graph = builder.compile(store=store)

In [128]:
print("Final answer")

Final answer


In [131]:
for message, metadata in graph.stream(
    input={"messages": [{"role": "user", "content": "which food do i like?"}]}, 
    stream_mode="messages"
):
   
    print(message.content, end=" ")

My message or my query content='which food do i like?' additional_kwargs={} response_metadata={} id='b1f68b73-7ad5-4772-b0bb-b14d62f21eb1'
my memories I prefer italina food
I love japaneese food
I dont like spicy food
 I love pizza
I love apples
 You  like  Italian  food ,  Japanese  food ,  pizza ,  and  apples .  You  also  do  not  like  spicy  food .  